In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Bidirectional, MaxPool1D
from tensorflow import nn
import pandas as pd
import numpy as np

2023-04-19 11:10:18.620233: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 데이터 로드

- 이때, mfcc_scaled 는 음성의 mfcc feature값을 의미
- text_tokenize는 텍스트를 konlpy로 okt()를 사용해 토큰화시키고, 정수화시킨 텍스트 시퀀스를 의미
- temp + eda는 temperature 데이터(길이 80;80보다 길면 자르고 짧으면 zero padding), eda 데이터(길이 80;80보다 길면 자르고 짧으면 zero padding)를 concat한 것을 의미함

In [1]:
import pickle
with open('../audio_augmented_per3500.pkl',"rb") as fr: #train 로드
    train = pickle.load(fr)
train

,index,Segment ID,mfcc_scaled,text_tokenize,temp+eda,sentiment_x
0,0,Sess13_script06_User026F_019,"[[0.0, 0.9999999999999999, 0.9608148846764442,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
1,1,Sess13_script06_User026F_019,"[[0.0, 0.8893497060147615, 0.9251501458319147,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
2,2,Sess13_script06_User026F_019,"[[0.0, 0.9672434656487046, 0.9493295501081099,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
3,3,Sess13_script06_User026F_019,"[[0.0, 0.8324701301043231, 0.8910219154763063,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
4,4,Sess13_script06_User026F_019,"[[0.0, 0.8692361742952204, 0.8943141198396264,...","[331, 29, 9462, 54, 4, 132, 70, 49, 3392, 9463...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",6
...,...,...,...,...,...,...
24495,7379,Sess35_script04_User070F_008,"[[0.032945521331930516, 0.40808203802457316, 0...","[42, 262, 1, 2859, 326, 32, 2859, 100, 344, 11...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",0
24496,7381,Sess31_script06_User061M_026,"[[0.0, 0.489686164522752, 0.8910410850529435, ...","[1819, 224, 248, 12, 1140, 0, 0, 0, 0, 0, 0, 0...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",0
24497,7383,Sess34_script02_User068M_020,"[[0.0012534706618411162, 0.4571524437803839, 0...","[534, 3934, 3934, 46, 1377, 4, 1618, 0, 0, 0, ...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",0
24498,7387,Sess27_script06_User054M_001,"[[0.09887480369794877, 0.5645720231907226, 0.7...","[15, 87, 722, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....",0


In [4]:
for i in range(0,len(train['mfcc_scaled'])):
    train['mfcc_scaled'][i]=np.mean(train['mfcc_scaled'][i],axis=1).reshape(600,1) #음성 데이터는 각 시퀀스끼리 mean 연산 진행

/tmp/ipykernel_16411/2951617071.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['mfcc_scaled'][i]=np.mean(train['mfcc_scaled'][i],axis=1).reshape(600,1)


In [5]:
import pickle
with open('../valid.pkl',"rb") as fr: #valid 로드
    valid = pickle.load(fr)
valid

,Segment ID,temp+eda,text_tokenize,mfcc_scaled,sentiment_x
0,Sess13_script04_User026F_025,"[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[70, 5, 9, 476, 239, 10, 75, 76, 29, 36, 62, 1...","[[0.01996256570538113, 0.5113530266302106, 0.8...",0
1,Sess28_script06_User055M_008,"[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[42, 3, 2227, 43, 27, 1, 136, 3274, 43, 27, 1,...","[[0.26915004084752314, 0.5238499532268169, 0.6...",0
2,Sess16_script03_User031M_036,"[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[49, 548, 1895, 17, 145, 14, 53, 659, 385, 13,...","[[0.17421271778196556, 0.4518614135726941, 0.6...",0
3,Sess07_script04_User014M_022,"[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[11, 25, 188, 188, 213, 28, 212, 11, 568, 1, 3...","[[0.2582311990368862, 0.5506098383515655, 0.56...",0
4,Sess10_script03_User020M_006,"[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[15, 23, 68, 30, 356, 2699, 2, 163, 296, 0, 0,...","[[0.009576183959700835, 0.398568994125834, 0.8...",1
...,...,...,...,...,...
1261,Sess13_script06_User026F_007,"[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[58, 736, 260, 2, 2399, 3389, 4213, 3197, 9437...","[[0.0, 0.46085726652280984, 0.7319667002532991...",4
1262,Sess01_script04_User002M_023,"[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[96, 19, 4, 383, 205, 334, 0, 0, 0, 0, 0, 0, 0...","[[0.0, 0.5498014295041825, 0.5528216368277, 0....",0
1263,Sess30_script02_User059F_029,"[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[105, 15, 148, 224, 502, 30, 3231, 0, 0, 0, 0,...","[[0.05568184697416323, 0.7497303486643664, 0.2...",0
1264,Sess33_script01_User066M_036,"[35.07, 35.07, 35.07, 35.07, 35.07, 35.07, 35....","[13402, 479, 85, 375, 177, 0, 0, 0, 0, 0, 0, 0...","[[0.19970357747630973, 0.4442398615317983, 0.7...",1


In [6]:
for i in range(0,len(valid['mfcc_scaled'])):
    valid['mfcc_scaled'][i]=np.mean(valid['mfcc_scaled'][i],axis=1).reshape(600,1) #음성 데이터는 각 시퀀스끼리 mean 연산 진행

/tmp/ipykernel_16411/3701760537.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['mfcc_scaled'][i]=np.mean(valid['mfcc_scaled'][i],axis=1).reshape(600,1)


In [7]:
import pickle
with open('../test.pkl',"rb") as fr: #test 로드
    test = pickle.load(fr)
test=test.reset_index()

In [8]:
for i in range(0,len(test['mfcc_scaled'])):
    test['mfcc_scaled'][i]=np.mean(test['mfcc_scaled'][i],axis=1).reshape(600,1) #음성 데이터는 각 시퀀스끼리 mean 연산 진행

/tmp/ipykernel_16411/468462434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mfcc_scaled'][i]=np.mean(test['mfcc_scaled'][i],axis=1).reshape(600,1)


In [9]:
y_train=train['sentiment_x']

In [10]:
y_valid=valid['sentiment_x']

In [11]:
y_test=test['sentiment_x']

# audio, text 임베딩

In [12]:
import tensorflow as tf
class a_TokenAndPositionEmbedding(layers.Layer):#오디오 임베딩
    def __init__(self, maxlen, embed_dim):
        super().__init__()
        self.a1_emb = tf.keras.layers.Conv1D(filters=32, kernel_size=5,padding='valid', activation='relu') #conv1d 연산 2번 진행
        self.a2_emb = tf.keras.layers.Conv1D(filters=32, kernel_size=5,padding='valid', activation='relu')
        self.a3_emb = MaxPool1D(pool_size=(12),padding='same') #maxpooling 진행
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim) 

    def call(self, x):
        maxlen = 50
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions) #포지셔널 임베딩
        x = self.a1_emb(tf.convert_to_tensor(x))
        x = self.a2_emb(x)
        x = self.a3_emb(x)
        
        return x + positions #포지셔널 임베딩과 음성 임베딩 값 더해주기

class t_TokenAndPositionEmbedding(layers.Layer): #텍스트 임베딩
    def __init__(self, maxlen,vocab_size, embed_dim):
        super().__init__()
        self.t_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim) #텍스트 임베딩
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen=50
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)

        x = self.t_emb(x) 
        return x + positions #포지셔널 임베딩과 텍스트 임베딩 값 더해주기
    
class bio_TokenAndPositionEmbedding(layers.Layer): #바이오 임베딩
    def __init__(self, maxlen, embed_dim):
        super().__init__()
        
        self.bio_temp_emb = layers.Embedding(input_dim=80, output_dim=embed_dim) #바이오 temp(온도) 임베딩
        self.bio_eda_emb = layers.Embedding(input_dim=80, output_dim=embed_dim) #바이오 eda 임베딩
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = 160
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)

        y1 = self.bio_temp_emb(tf.slice(x,[0,0],[-1,80])) 
        y2= self.bio_eda_emb(tf.slice(x,[0,80],[-1,-1]))

        x=tf.concat([y1,y2],axis=1) #temp 임베딩과 eda 임베딩은 concat 진행
        

        return x + positions #포지셔널 임베딩과 바이오 임베딩 값 더해주기


# audio-text 트랜스포머 block 및 multihead attention (쿼리가 텍스트, 키 밸류는 오디오), 타겟:텍스트, 소스: 오디오

텍스트 데이터에 오디오 데이터의 정보가 입혀지는 과정

In [14]:
class at_MultiHeadAttention(tf.keras.layers.Layer):  #오디오->텍스트 멀티헤드어텐션
    def __init__(self, embedding_dim, num_heads=8):
        super(at_MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs_text, inputs_audio): 
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs_text)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs_text)
        key = self.key_dense(inputs_audio)
        value = self.value_dense(inputs_audio)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs
    
class at_co_TransformerBlock(layers.Layer): #audio->text cross attention이 이뤄지는 트랜스포머 블럭
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = at_MultiHeadAttention(embedding_dim=32, num_heads=4)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs_text, inputs_audio, training):
        attn_output = self.att(inputs_text, inputs_audio)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs_text + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# text-audio 트랜스포머 block 및 multihead attention : (쿼리가 오디오, 키 밸류는 텍스트), 타겟:오디오, 소스: 텍스트
오디오 데이터에 텍스트 정보가 입혀지는 과정

In [21]:

class ta_MultiHeadAttention(tf.keras.layers.Layer): #텍스트->오디오 멀티헤드어텐션
    def __init__(self, embedding_dim, num_heads=8):
        super(ta_MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs_audio, inputs_text): 
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs_audio)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs_audio)
        key = self.key_dense(inputs_text)
        value = self.value_dense(inputs_text)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs
    
class ta_co_TransformerBlock(layers.Layer): #text->audio cross attention이 이뤄지는 트랜스포머 블럭
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = ta_MultiHeadAttention(embedding_dim=32, num_heads=4)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs_audio, inputs_text, training):
        attn_output = self.att(inputs_audio, inputs_text)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs_audio + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)

        return self.layernorm2(out1 + ffn_output)

# 트랜스포머
바이오는 cross-attetnion이 아니고 그냥 concat해서 input되기때문에 그냥 트랜스포머 block을 사용

In [22]:


class TransformerBlock(layers.Layer):  #공통적인 트랜스포머 블럭
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadAttention(embedding_dim=embed_dim, num_heads=num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        
        return self.layernorm2(out1 + ffn_output)

In [23]:
class MultiHeadAttention(tf.keras.layers.Layer): #공통적인 트랜스포머 멀티헤드어텐션
    def __init__(self, embedding_dim, num_heads=4):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim 
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

# 멀티모달 트랜스포머

In [29]:
embed_dim = 32 # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen=160
training=1

audio_input = layers.Input(shape=(600,1))#audio_input = layers.Input(shape=(600,50)) #
text_input = layers.Input(shape=(50,)) #
bio_input = layers.Input(shape=(160,)) #


a_embedding_layer = a_TokenAndPositionEmbedding(50, 32) #오디오 임베딩
audio_embedding = a_embedding_layer(audio_input)
t_embedding_layer = t_TokenAndPositionEmbedding(50,14991, embed_dim) #텍스트 임베딩
text_embedding = t_embedding_layer(text_input)  

b_embedding_layer = bio_TokenAndPositionEmbedding(160, embed_dim) #바이오 데이터(temp+eda) 임베딩
bio_embedding = b_embedding_layer(tf.convert_to_tensor(bio_input))  




# 멀티모달 cross-attention
at_transformer_block = at_co_TransformerBlock(32, 4, 32) #audio-text transformer
ta_transformer_block = ta_co_TransformerBlock(embed_dim, num_heads, ff_dim) #text-audio transformer
bio_transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim) # bio eda,temp transformer
transformer_block = TransformerBlock(96, num_heads, ff_dim)


#진행되는 부분
output_at = at_transformer_block(text_embedding, audio_embedding) #audio-text transformer
output_ta = ta_transformer_block(audio_embedding, text_embedding) #text-audio transformer
output_bio = bio_transformer_block(bio_embedding) # bio eda,temp transformer
output_at = layers.GlobalAveragePooling1D()(output_at)
output_ta = layers.GlobalAveragePooling1D()(output_ta)
output_bio = layers.GlobalAveragePooling1D()(output_bio)

concat_embedding = layers.concatenate([output_at, output_ta, output_bio]) #세가지 모달리티 concat 진행
output = transformer_block(concat_embedding)  #공통 트랜스포머 블럭

x = layers.GlobalAveragePooling1D()(output)
x = layers.Dense(80, activation="sigmoid")(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="sigmoid")(x)
x = layers.Dropout(0.1)(x)


outputs = layers.Dense(7, activation="softmax")(x)  #클래시파이어(분류기)


model = keras.Model(inputs=[audio_input, text_input, bio_input], outputs=outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

# 모델 학습

In [30]:
history = model.fit(
    [[np.array(train['mfcc_scaled'].to_list()),np.array(train['text_tokenize'].to_list()),np.array(train['temp+eda'].to_list())]], y_train, batch_size=16, epochs=5, validation_data=([[np.array(valid['mfcc_scaled'].to_list()),np.array(valid['text_tokenize'].to_list()),np.array(valid['temp+eda'].to_list())]], y_valid)
)


Epoch 1/5
1532/1532 [==============================] - 234s 149ms/step - loss: 1.3182 - accuracy: 0.4900 - val_loss: 0.8006 - val_accuracy: 0.8057
Epoch 2/5
1532/1532 [==============================] - 224s 146ms/step - loss: 0.4780 - accuracy: 0.8678 - val_loss: 0.7403 - val_accuracy: 0.8104
Epoch 3/5
1532/1532 [==============================] - 210s 137ms/step - loss: 0.2479 - accuracy: 0.9264 - val_loss: 0.7411 - val_accuracy: 0.8144
Epoch 4/5
1532/1532 [==============================] - 225s 147ms/step - loss: 0.1794 - accuracy: 0.9379 - val_loss: 0.8427 - val_accuracy: 0.7773
Epoch 5/5
1532/1532 [==============================] - 230s 150ms/step - loss: 0.1366 - accuracy: 0.9496 - val_loss: 0.8469 - val_accuracy: 0.7852


# 테스트 데이터셋으로 예측

In [31]:
y_pred=model.predict([np.array(test['mfcc_scaled'].to_list()),np.array(test['text_tokenize'].to_list()),np.array(test['temp+eda'].to_list())])
yp=[]
for i in y_pred:
    yp.append(i.argmax())

81/81 [==============================] - 3s 24ms/step


In [33]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test.to_list(), yp))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test.to_list(), yp, output_dict=True, digits=5)) 

[[1906   75   16   42   13    7    3]
 [ 305   37    2   13    2    2    0]
 [  26    0    2    1    0    0    0]
 [  41    2    0    4    1    0    0]
 [  29    1    0    1    5    0    0]
 [  13    0    1    0    1    1    0]
 [   9    0    0    2    0    0    4]]
{'0': {'precision': 0.8183769858308286, 'recall': 0.9243452958292919, 'f1-score': 0.868139375996356, 'support': 2062}, '1': {'precision': 0.3217391304347826, 'recall': 0.10249307479224377, 'f1-score': 0.15546218487394958, 'support': 361}, '2': {'precision': 0.09523809523809523, 'recall': 0.06896551724137931, 'f1-score': 0.08, 'support': 29}, '3': {'precision': 0.06349206349206349, 'recall': 0.08333333333333333, 'f1-score': 0.07207207207207209, 'support': 48}, '4': {'precision': 0.22727272727272727, 'recall': 0.1388888888888889, 'f1-score': 0.1724137931034483, 'support': 36}, '5': {'precision': 0.1, 'recall': 0.0625, 'f1-score': 0.07692307692307693, 'support': 16}, '6': {'precision': 0.5714285714285714, 'recall': 0.266666666

증강 전 데이터를 사용하면 오버피팅이 매우 크고, 모델이 중립 감정만 찍는 경향이 있음 

- 증강을 통해 모든 감정에 대해 모델이 잘 식별할 수 있도록 학습을 하고, sklearn 의 metrics를 활용하여 모든 감정에 대해 f1score를 구할 수 있었음

- test 데이터셋은 랜덤 8개의 세션으로 구성된 데이터 셋임
- test 데이터셋의 감정 분포도 중립이 타 감정(ex: 행복)에 비해 매우 많으므로 macro score가 아닌, weigthed score로 평가 진행
- 최종스코어는 아래와 같음
- 'accuracy': 0.7631476431632256,'precision': 0.7120389331643536, 'recall': 0.7631476431632256, 'f1-score': 0.7264888635529725 의 성능을 낼 수 있었음.